In [1]:
import httpx
from moveon import MoveonClient

In [2]:
client = MoveonClient(
    "https://urfu02-api.moveonru.com/restService/index.php?version=3.0",
    httpx.AsyncClient(cert=("moveon.cert", "moveon.key")),
)

In [3]:
def list_contacts_template(
    visible_columns="",
   contact_id = None,
   institution_id=None
):
    rules = []
    if contact_id:
        rules.append({
            "field": "contact.id",
            "op": "eq",
            "data": contact_id
        })
    if institution_id:
        rules.append({
            "field": "contact.institution_id",
            "op": "eq",
            "data": institution_id
        })

    return {
        "entity": "contact",
        "action": "list",
        "data": {
            "filters": {
                "groupOp": "AND",
                "rules": rules,
            },
            "visibleColumns": visible_columns,
            "locale": "eng",
            "sidx": "contact.id",
            "sord": "asc",
            "_search": "true" if len(rules) > 0 else "false",
            "page": 1,
            "rows": 100,
        },
    }

In [4]:
# Все поля contact
response = await client.queue_and_get(list_contacts_template())
response["data"]["availableKeys"].split(";")

['contact.id',
 'contact.person_id',
 'contact.first_name',
 'contact.surname',
 'contact.name',
 'contact.function',
 'contact.remarks',
 'contact.responsibility',
 'contact.department',
 'contact.group',
 'contact.territory',
 'contact.created_on',
 'contact.is_active',
 'contact.created_by',
 'contact.last_modified_on',
 'contact.last_modified_by',
 'contact.institution_id',
 'contact.institution',
 'customfield3',
 'customfield4',
 'customfield5',
 'customfield6',
 'customfield7',
 'customfield8',
 'customfield9',
 'customfield10',
 'customfield11',
 'customfield12',
 'customfield13',
 'customfield14',
 'customfield15',
 'customfield16',
 'customfield17',
 'customfield18',
 'customfield19',
 'customfield20',
 'customfield21',
 'customfield22',
 'customfield23',
 'customfield24',
 'customfield25',
 'customfield26',
 'customfield27',
 'customfield28',
 'customfield29',
 'customfield30',
 'customfield31',
 'customfield32',
 'customfield33',
 'customfield34',
 'customfield35',
 'custom

In [4]:
async def find_contacts(contact_id=None, institution_id=None):
    contact_list = []
    async for page in client.iter_pages(
        list_contacts_template(
            visible_columns="contact.id;contact.name;contact.institution_id;contact.institution",
            contact_id=contact_id,
            institution_id=institution_id,
        )
    ):
        for row in page["data"]["rows"]:
            contact_list.append(row)

    return contact_list

In [6]:
await find_contacts(institution_id=2880)

[{'contact.id': '4286',
  'contact.name': 'Arja, Rosenholm',
  'contact.institution_id': '2880',
  'contact.institution': 'Tampere University'},
 {'contact.id': '4287',
  'contact.name': 'Elise, Feron',
  'contact.institution_id': '2880',
  'contact.institution': 'Tampere University'}]

In [7]:
async def update_contact(contact_id, new_institution_id):
    return await client.queue_and_get({
        "entity": "contact",
        "action": "save",
        "data": {
            "entity": "contact",
            "contact.id": contact_id,
            "contact.institution_id": new_institution_id
        }
    })

In [8]:
# Название института обновляется автоматически при смене id
await update_contact(contact_id=4286, new_institution_id=2883)

{'data': {'id': '4286',
  'availableKeys': 'contact.id;contact.person_id;contact.first_name;contact.surname;contact.gender_id;contact.function;contact.phone;contact.phone_2;contact.mobile;contact.fax;contact.remarks;contact.title;contact.responsibility;contact.department;contact.communication_language;contact.email;contact.groups;contact.is_active;contact.is_deleted;contact.institution_id;contact.action_date;contact.user_id'}}

In [5]:
await find_contacts(contact_id=4286)

[{'contact.id': '4286',
  'contact.name': 'Arja, Rosenholm',
  'contact.institution_id': '2883',
  'contact.institution': 'Tampere University'}]